In [133]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


!pip install surprise
from surprise import Reader, Dataset
from surprise import SVD, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, SVDpp
from surprise.model_selection import GridSearchCV
from surprise import accuracy

In [134]:
from google.colab import drive
drive.mount('/content/drive')
data_read = pd.read_csv('/content/drive/My Drive/LHL_Final_Project/Written_Reviews/Beer_Recommendations_EDA_V01.csv')
data_read.shape

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(853486, 19)

User 10% of the dataset for testing

In [135]:
df = data_read.sample(frac=0.10, random_state=42)
df.shape

(85349, 19)

In [136]:
df = df.rename(columns={'username': 'userID','Name':'itemID','user_overall_score':'rating'})
#df = df.fillna(0)
df_copy = df.copy()
df.tail(10)

,beer_id,userID,date,text,look,smell,taste,feel,rating,avg_score,beer_name,style,abv,brewery_id,brewery_name,country,count,style_count,itemID
484732,48911,TurdFurgison,2009-04-04,"I bought a 4-pack for $11.99 in Columbus, firs...",4.0,4.50,3.5,4.0,3.50,3.82,Cerise,Fruit and Field Beer,6.5,1199,Founders Brewing Company,US,1,15389,Founders Brewing Company Cerise
788422,33832,angelmonster,2009-03-16,Pours a 2 finger dark chocolate head. Body is ...,4.0,4.00,4.0,4.5,3.50,3.95,Palo Santo Marron,American Brown Ale,12.0,64,Dogfish Head Craft Brewery,US,1,14014,Dogfish Head Craft Brewery Palo Santo Marron
209053,43905,Tilley4,2011-02-12,Finally! Been looking for this one for a while...,4.0,4.50,4.5,4.5,4.50,4.47,CaliBelgique IPA,Belgian IPA,6.9,147,Stone Brewing,US,1,6766,Stone Brewing CaliBelgique IPA
833477,55081,OttoBomb,2010-03-11,22 oz bomber poured into Dogfish Head signatur...,4.5,4.50,4.0,4.5,4.50,4.30,Ranger,American IPA,6.5,192,New Belgium Brewing,US,1,83478,New Belgium Brewing Ranger
148041,299,mltobin,2008-01-18,"Poured into a glass, a medium sized head forms...",4.0,4.00,4.0,4.0,3.50,3.90,9,Fruit and Field Beer,5.1,96,Magic Hat Brewing Company,US,1,15389,Magic Hat Brewing Company 9
273289,5428,ppoitras,2006-06-03,22oz bottle obtained via trade with CRJMellor....,4.5,4.50,4.5,4.5,4.50,4.50,Dragons Milk,American Imperial Stout,11.0,335,New Holland Brewing Company,US,1,46108,New Holland Brewing Company Dragons Milk
702910,139515,woodychandler,2016-12-01,"Rollin', rollin', rollin', keep those bottles ...",4.0,3.75,3.5,3.5,3.75,3.64,Samuel Adams Rebel Rouser Double IPA,American Imperial IPA,8.4,35,Boston Beer Company Samuel Adams,US,1,61144,Boston Beer Company Samuel Adams Samuel Adams ...
469238,32423,tempest,2010-10-20,I split a large bottle at Monk's Cafe for $15....,4.0,4.00,4.0,4.0,4.50,4.10,Red Barn Ale,Belgian Saison,6.7,18149,The Lost Abbey,US,1,18552,The Lost Abbey Red Barn Ale
661008,10672,zdk9,2010-02-01,Michigan Trip Beer #1- An amazing find on my s...,4.5,4.50,5.0,5.0,5.00,4.85,Bourbon County Brand Stout,American Imperial Stout,14.1,1146,Goose Island Beer Co,US,1,46108,Goose Island Beer Co Bourbon County Brand Stout
473544,36728,pants678,2009-12-26,"Opaque black, beige head in a stander snifter....",4.5,5.00,4.5,4.5,3.00,4.32,10 Commandments,Belgian Strong Dark Ale,11.0,18149,The Lost Abbey,US,1,23267,The Lost Abbey 10 Commandments


In [137]:
user_beer_rating = df.pivot_table(index='userID', columns='itemID', values='rating')
user_beer_rating

itemID,21st Amendment Brewery Back In Black,21st Amendment Brewery Brew Free Or Die IPA,21st Amendment Brewery Fireside Chat,21st Amendment Brewery Hell Or High Watermelon Wheat Beer,21st Amendment Brewery Hop Crisis,21st Amendment Brewery Monks Blood,3 Floyds Brewing Co Alpha King,3 Floyds Brewing Co Alpha Klaus Christmas Xmas Porter,3 Floyds Brewing Co Apocalypse Cow,3 Floyds Brewing Co Arctic Panzer Wolf,3 Floyds Brewing Co Behemoth Blonde Barleywine,3 Floyds Brewing Co Black Sun Stout,3 Floyds Brewing Co Blackheart,3 Floyds Brewing Co Broo Doo,3 Floyds Brewing Co Dark Lord Imperial Stout,3 Floyds Brewing Co Dreadnaught IPA,3 Floyds Brewing Co Gumballhead,3 Floyds Brewing Co Marshmallow Handjee,3 Floyds Brewing Co Moloko,3 Floyds Brewing Co Permanent Funeral,3 Floyds Brewing Co Pride Joy Mild Ale,3 Floyds Brewing Co Rabbid Rabbit,3 Floyds Brewing Co Robert The Bruce,3 Floyds Brewing Co Zombie Dust,Abbaye de Leffe SA Leffe Blonde,Abbaye de Leffe SA Leffe Brune Brown,Abita Brewing Co Amber,Abita Brewing Co Andygator,Abita Brewing Co Jockamo IPA,Abita Brewing Co Mardi Gras Bock,Abita Brewing Co Pecan Ale,Abita Brewing Co Purple Haze,Abita Brewing Co SOS Save Our Shore,Abita Brewing Co Strawberry Harvest Lager,Abita Brewing Co Turbodog,Adnams plc Sole Bay Brewery Adnams Broadside,Alaskan Brewing Co Alaskan Amber,Alaskan Brewing Co Alaskan Icy Bay IPA,Alaskan Brewing Co Alaskan Kolsch,Alaskan Brewing Co Alaskan Smoked Porter,...,Weyerbacher Brewing Co Double Simcoe IPA,Weyerbacher Brewing Co Imperial Pumpkin Ale,Weyerbacher Brewing Co Insanity Oak BarrelAged,Weyerbacher Brewing Co Merry Monks,Weyerbacher Brewing Co Quad,Weyerbacher Brewing Co Riserva,Weyerbacher Brewing Co Sunday Morning Stout,Weyerbacher Brewing Co TINY,Weyerbacher Brewing Co Verboten,Weyerbacher Brewing Co Weyerbacher Wit,Weyerbacher Brewing Co Winter Ale,Whitbread PLC Mackeson Triple XXX Stout,Widmer Brothers Brewing Company Brrr,Widmer Brothers Brewing Company Drifter Pale Ale,Widmer Brothers Brewing Company Nelson Imperial IPA,Widmer Brothers Brewing Company Pitch Black IPA,Widmer Brothers Brewing Company Widmer Hefeweizen,Williams Brothers Brewing Company Alba Scots Pine Ale,Williams Brothers Brewing Company Ebulum Elderberry Black Ale,Williams Brothers Brewing Company Fraoch Heather Ale,Williams Brothers Brewing Company Grozet,Williams Brothers Brewing Company Kelpie Seaweed Ale,Wychwood Brewery Company Ltd Bah Humbug Christmas Cheer Ale,Wychwood Brewery Company Ltd Hobgoblin,Würzburger Hofbräu AG Julius Echter HefeWeissbier Dunkel,Würzburger Hofbräu AG Julius Echter HefeWeissbier Hell,Yards Brewing Co Brawler Pugilist Style Ale,Yards Brewing Co Extra Special Ale ESA,Yards Brewing Co India Pale Ale,Yards Brewing Co Jeffersons Golden Ale,Yards Brewing Co Philadelphia Pale Ale,Yards Brewing Co Washingtons Porter,Yazoo Brewing Company Sue,Yuengling Brewery Lord Chesterfield Ale,Yuengling Brewery Yuengling Black Tan,Yuengling Brewery Yuengling Dark Brewed Porter,Yuengling Brewery Yuengling Light Lager,Yuengling Brewery Yuengling Traditional Lager,Zywiec Breweries PLC Heineken Porter,Zywiec Breweries PLC Heineken Zywiec Beer
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-Rick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0110x011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
05Harley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [138]:
test_ratings = user_beer_rating["Yards Brewing Co Washingtons Porter"]
beer_like_test = user_beer_rating.corrwith(test_ratings)
beer_like_test

itemID
21st Amendment Brewery Back In Black                         1.0
21st Amendment Brewery Brew Free Or Die IPA                  NaN
21st Amendment Brewery Fireside Chat                         NaN
21st Amendment Brewery Hell Or High Watermelon Wheat Beer    NaN
21st Amendment Brewery Hop Crisis                            NaN
                                                            ... 
Yuengling Brewery Yuengling Dark Brewed Porter               NaN
Yuengling Brewery Yuengling Light Lager                      NaN
Yuengling Brewery Yuengling Traditional Lager                NaN
Zywiec Breweries PLC Heineken Porter                         NaN
Zywiec Breweries PLC Heineken Zywiec Beer                    NaN
Length: 1709, dtype: float64

In [139]:
corr_test = pd.DataFrame(beer_like_test, columns=['Correlation'])
corr_test.dropna(inplace=True)
corr_test.sort_values(by = 'Correlation',ascending= False).head(10)

,Correlation
itemID,
Yards Brewing Co Washingtons Porter,1.0
Bells Brewery Eccentric Café General Store Rye Stout,1.0
Dogfish Head Craft Brewery Raison DExtra,1.0
Dogfish Head Craft Brewery World Wide Stout,1.0
Brouwerij Duvel Moortgat NV Duvel Belgian Golden Ale,1.0
Brouwerij De Dolle Brouwers De Dolle Dulle Teve Mad Bitch,1.0
Genesee Brewing Co Dundee Brewing Co Genesee Cream Ale,1.0
Great Divide Brewing Company Saint Bridgets Porter,1.0
Greene King Morland Brewery Old Speckled Hen,1.0


In [140]:
newDataFrameTest = pd.DataFrame(df.groupby('itemID')['rating']\
.agg(['count','mean'])\
.sort_values(['count', 'mean'], ascending=[False, True]))

In [141]:
newDataFrameTest = newDataFrameTest.rename({'count':'vote_count',\
                         'mean':'vote_average'}, axis='columns')
newDataFrameTest

,vote_count,vote_average
itemID,,
Dogfish Head Craft Brewery 90 Minute IPA,247,4.115385
Sierra Nevada Brewing Co Celebration Fresh Hop IPA,223,4.144619
Founders Brewing Company Breakfast Stout,223,4.393498
North Coast Brewing Co Old Rasputin,220,4.193182
Brasserie de Rochefort Trappistes Rochefort 10,217,4.334101
...,...,...
BrewDog 5AM Saint,13,3.846154
OFallon Brewery 5Day IPA,13,3.884615
Sixpoint Brewery 3Beans,13,3.942308


In [142]:
from collections import defaultdict

def top_rated_beer(predictions, numberOfPredictions=10):

    top_predictions = defaultdict(list)
    for user_id, iid, true_r, est, _ in predictions:
        top_predictions[user_id].append((iid, est))

    for user_id, user_ratings in top_predictions.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_predictions[user_id] = user_ratings[:numberOfPredictions]

    return top_predictions


In [143]:
reader = Reader(rating_scale=(0, 5))
df = Dataset.load_from_df(df[['userID', 	'itemID', 	'rating' 	]], reader=reader)

In [144]:
# Creating a train/test set within Surprise
raw_ratings = df.raw_ratings
threshold = int(.7 * len(raw_ratings))                                     
trainset_raw_ratings = raw_ratings[:threshold]                             
test_raw_ratings = raw_ratings[threshold:]  
df.raw_ratings = trainset_raw_ratings

In [145]:
param_grid = {
    'n_factors':[5, 10,20],
    'n_epochs': [5, 10, 20], 
    'lr_all': [0.002, 0.005],
    'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(df)

In [146]:
model = gs.best_estimator['rmse']
trainset = df.build_full_trainset()
model.fit(trainset)
print('algorithm: SVD with GridSearchCV')
# Trainset
predictions = model.test(trainset.build_testset())
print('Biased accuracy on trainset,', end='   ')
accuracy.rmse(predictions)
# Testset
testset = df.construct_testset(test_raw_ratings)
predictions = model.test(testset)
print('Unbiased accuracy on testset,', end=' ')
accuracy.rmse(predictions)

algorithm: SVD with GridSearchCV
Biased accuracy on trainset,   RMSE: 0.5080
Unbiased accuracy on testset, RMSE: 0.5381


0.5380643859481159

In [147]:
# knn = KNNBasic()
# trainset = df.build_full_trainset()
# knn.fit(trainset)

# print('algorithm: KNNBasic')
# # Trainset
# predictions = knn.test(trainset.build_testset())
# print('Biased accuracy on trainset,', end='   ')
# accuracy.rmse(predictions)

# # Testset
# testset = df.construct_testset(test_raw_ratings)
# predictions = knn.test(testset)
# print('Unbiased accuracy on testset,', end=' ')
# accuracy.rmse(predictions)

In [148]:
# knn_baseline = KNNBaseline()
# trainset = df.build_full_trainset()
# knn_baseline.fit(trainset)

# print('algorithm: KNNBaseline')
# # Trainset
# predictions = knn_baseline.test(trainset.build_testset())
# print('Biased accuracy on trainset,', end='   ')
# accuracy.rmse(predictions)

# # Testset
# testset = df.construct_testset(test_raw_ratings)
# predictions = knn_baseline.test(testset)
# print('Unbiased accuracy on testset,', end=' ')
# accuracy.rmse(predictions)

In [149]:
# knn_with_mean = KNNWithMeans()
# trainset = df.build_full_trainset()
# knn_with_mean.fit(trainset)

# print('algorithm: KNNWithMeans')
# # Trainset
# predictions = knn_with_mean.test(trainset.build_testset())
# print('Biased accuracy on trainset,', end='   ')
# accuracy.rmse(predictions)

# # Testset
# testset = df.construct_testset(test_raw_ratings)
# predictions = knn_with_mean.test(testset)
# print('Unbiased accuracy on testset,', end=' ')
# accuracy.rmse(predictions)

In [150]:
# knn_with_z = KNNWithZScore()
# trainset = df.build_full_trainset()
# knn_with_z.fit(trainset)

# print('algorithm: KNNWithZScore')
# # Trainset
# predictions = knn_with_z.test(trainset.build_testset())
# print('Biased accuracy on trainset,', end='   ')
# accuracy.rmse(predictions)

# # Testset
# testset = df.construct_testset(test_raw_ratings)
# predictions = knn_with_z.test(testset)
# print('Unbiased accuracy on testset,', end=' ')
# accuracy.rmse(predictions)

In [151]:
svdpp = SVDpp()
trainset = df.build_full_trainset()
svdpp.fit(trainset)

print('algorithm: SVDpp')
# Trainset
predictions = svdpp.test(trainset.build_testset())
print('Biased accuracy on trainset,', end='   ')
accuracy.rmse(predictions)

# Testset
testset = df.construct_testset(test_raw_ratings)
predictions = svdpp.test(testset)
print('Unbiased accuracy on testset,', end=' ')
accuracy.rmse(predictions)

algorithm: SVDpp
Biased accuracy on trainset,   RMSE: 0.4610
Unbiased accuracy on testset, RMSE: 0.5337


0.5337204494237419

In [152]:
# param_grid = {
#     'n_factors':[5, 10,20],
#     'n_epochs': [5, 10, 20], 
#     'lr_all': [0.002, 0.005],
#     'reg_all': [0.4, 0.6]}
# gs_svdpp = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)
# gs_svdpp.fit(df)

# model_svdpp = gs_svdpp.best_estimator['rmse']
# trainset = df.build_full_trainset()
# model_svdpp.fit(trainset)
# print('algorithm: SVDpp with GridSearchCV')
# # Trainset
# predictions = model_svdpp.test(trainset.build_testset())
# print('Biased accuracy on trainset,', end='   ')
# accuracy.rmse(predictions)
# # Testset
# testset = df.construct_testset(test_raw_ratings)
# predictions = model_svdpp.test(testset)
# print('Unbiased accuracy on testset,', end=' ')
# accuracy.rmse(predictions)

Section Modified From https://www.jiristodulka.com/post/recsys_cf/

In [153]:
param_grid = {'n_factors': [4,6,9,11,14,18,29]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)
gs.fit(df)


# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.5359182333271367
{'n_factors': 4}


In [154]:
!pip install streamlit
!pip install pyngrok

In [163]:
%%writefile app.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from surprise import Reader, Dataset
from surprise import SVD, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, SVDpp
from surprise.model_selection import GridSearchCV
from collections import defaultdict
from surprise import accuracy

from google.colab import drive
drive.mount('/content/drive')
data_read = pd.read_csv('/content/drive/My Drive/LHL_Final_Project/Written_Reviews/Beer_Recommendations_EDA_V01.csv')
df = data_read.sample(frac=0.10, random_state=42)
df = df.rename(columns={'username': 'userID','Name':'itemID','user_overall_score':'rating'})
df_copy = df.copy()
df_copy['Name'] = df_copy['brewery_name']+' '+df_copy['beer_name']
df_copy = df_copy[['userID', 'Name', 'itemID', 'style', 'rating']]

reader = Reader(rating_scale=(0, 5))
df = Dataset.load_from_df(df[['userID', 	'itemID', 	'rating' 	]], reader=reader)
# Creating a train/test set within Surprise
raw_ratings = df.raw_ratings
threshold = int(.7 * len(raw_ratings))                                     
trainset_raw_ratings = raw_ratings[:threshold]                             
test_raw_ratings = raw_ratings[threshold:]  
df.raw_ratings = trainset_raw_ratings

algo_SVD = SVD(n_factors = 4)
trainset = df.build_full_trainset()
algo_SVD.fit(trainset)
print('algorithm: SVD with GridSearchCV')
# Trainset
predictions = algo_SVD.test(trainset.build_testset())
print('Biased accuracy on trainset,', end='   ')
accuracy.rmse(predictions)
# Testset
testset = df.construct_testset(test_raw_ratings)

predictions = algo_SVD.test(testset)
print('Unbiased accuracy on testset,', end=' ')
accuracy.rmse(predictions)


def get_top_n(predictions, userID, df_copy, n = 10):
    '''Return the top N (default) itemID for a user,.i.e. userID and history for comparison
    Args:
    Returns: 
  
    '''
    #Part I.: Surprise docomuntation
    
    #1. First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    #2. Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_n[uid] = user_ratings[: n ]
    
    #Part II.: inspired by: https://beckernick.github.io/matrix-factorization-recommender/
    
    #3. Tells how many beer the user has already rated
    user_data = df_copy[df_copy.userID == (userID)]
    print('User {0} has already rated {1} beer.'.format(userID, user_data.shape[0]))

    
    #4. Data Frame with predictions. 
    preds_df = pd.DataFrame([(id, pair[0],pair[1]) for id, row in top_n.items() for pair in row],
                        columns=["userID" ,"itemID","rat_pred"])
    
    
    #5. Return pred_usr, i.e. top N recommended beer. 
    pred_usr = pd.merge(preds_df[preds_df["userID"] == (userID)],df_copy, on = 'itemID')
    pred_usr = pred_usr.rename(columns={'userID_x': 'userID'})
    pred_usr = pred_usr[['userID', 'itemID', 'style', 'rat_pred']]
            
    #6. Return hist_usr, i.e. top N historically rated beer.
    hist_usr = df_copy[['userID','itemID','style','rating']]
    hist_usr = hist_usr[hist_usr['userID'] == userID]
    
    
    return hist_usr, pred_usr

import streamlit as st
st.set_page_config(layout="wide") 
st.header('Recommendations for Existing Users')

# Sample of usernames for testing
st.write('Sample Usernames')
st.dataframe(df_copy['userID'].sample(20))

with st.form(key='form1'):  
  username = st.text_input('Select Username')
  st.form_submit_button(label = 'Press Here for New Beer!')
  hist_SVD, pred_SVD = get_top_n(predictions, df_copy = df_copy, userID = username)


st.write('Displaying Recommendations for User: ', username)

pred_SVD = pred_SVD.drop_duplicates('itemID')
hist_SVD = hist_SVD.drop_duplicates('itemID')
st.write('Recommendations')
st.dataframe(pred_SVD)
st.write('Previous Ratings')
st.dataframe(hist_SVD)

Overwriting app.py


In [164]:
!ls

app.py	drive  sample_data  user.py


# Insert ngrok authtoken

In [167]:
import os
!ngrok authtoken XXXXXXXXXXXXXXXXXXXXXXXXXX
from pyngrok import ngrok
!streamlit run --server.port 80 app.py&>/dev/null&
publ_url = ngrok.connect(port='80')
publ_url 

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


<NgrokTunnel: "http://8a5a-34-73-2-173.ngrok.io" -> "http://localhost:80">

In [169]:
ngrok.kill()